In [ ]:
import importlib
import numpy as np
import general_utils
import tampering_utils
import xai_utils
import generate_explanations, visualize_results
importlib.reload(general_utils)
importlib.reload(tampering_utils)
importlib.reload(xai_utils)
importlib.reload(generate_explanations)
importlib.reload(visualize_results)
from generate_explanations import *
from general_utils import train_and_save_model
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

In [ ]:
path_data = "data"
data_names = ["kaggle"]
max_len = 100
custom_dropouts = [None, 10, 20, 30, 40, 50]
batch_size = 64
seqs_count = 24
path = "all_results/results1/"

explain_for_dataset = {
    "kaggle": explain_for_kaggle,
    "AG": explain_for_ag,
    "toxic": explain_for_toxic
}

# 0 for training
# 1 for explaining
mode = 2

for data_name in data_names:
    data = load_tokenized_data(batch_size=batch_size, data_name=data_name, make_even_flag=True)
    if mode==0:
        train_and_save_model(path, max_len=max_len, path_data="data", data_name=data_name, 
                             custom_dropouts=custom_dropouts, batch_size=batch_size, epochs=5)
    elif mode==1:
        explain_for_dataset[data_name](data, seqs_count, path=path, custom_dropouts=custom_dropouts)

In [ ]:
# from visualize_results import show_for_kaggle

# #30, 4
# #None, [4,5,8,11,23]
# # 50, [2,23]

# indices = np.where(np.argmax(data[1], axis=1) == 0)[0]
# print(indices)
# indices = indices[np.where(indices<seqs_count)]
# contributions = show_for_kaggle(
#     path, 'kaggle', data, [4,5,8,11,23], ["lime"], custom_dropouts=([None])
# );

In [ ]:
save_figures = True
fig_acc = None

compute_accuracies(data, data_name=data_name, path=path)
fig_acc = plot_accuracies(data_name=data_name, path=path)

# if save_figures:
#     fig_acc.savefig(path+'Figures/accuracies_'+data_name+'_'+str(max_len)+'.pdf', layout='tight')
#     plt.close(fig_acc)

In [ ]:
importlib.reload(generate_explanations)
from generate_explanations import *


save_figures = False
figs_cs, figs_l1, figs_l2 = None, None, None

figs_cs, scores = plot_cosine_similarities(path=path, data_name=data_name, seqs_count=seqs_count)
# figs_l1 = plot_L_Norms(1, path=path, data_name=data_name)
# figs_l2 = plot_L_Norms(2, path=path, data_name=data_name)


if save_figures:
    with PdfPages(path+'Figures/'+"hybrid_"+data_name+"_"+str(max_len)+'cosine_similarity.pdf') as pdf:
        for fig in figs_cs:
            plt.figure(fig.number)
            pdf.savefig(layout="tight")
            plt.close(fig)

    with PdfPages(path+'Figures/'+"hybrid_"+data_name+"_"+str(max_len)+'L1_norms.pdf') as pdf:
        for fig in figs_l1:
            plt.figure(fig.number)
            pdf.savefig(layout="tight")
            plt.close(fig)
        

    with PdfPages(path+'Figures/'+"hybrid_"+data_name+"_"+str(max_len)+'L2_norms.pdf') as pdf:
        for fig in figs_l2:
            plt.figure(fig.number)
            pdf.savefig(layout="tight")
            plt.close(fig)

In [ ]:
save_figures = True
is_remove = False
num_words_s = [50, 90]#None, 10, 30, 40, 50]

for num_words in num_words_s:
    figs_da = None  
    figs_da = plot_descriptive_accuracies(data, path=path, data_name=data_name, seqs_count=seqs_count,
                                          num_words=num_words, is_remove=is_remove)

    if save_figures:
        with PdfPages(path+'Figures/'+"hybrid_"+data_name+"_"+str(max_len)+'_descriptive_accuracy_'+str(num_words)+'_'+str(is_remove)+'.pdf') as pdf:
            plt.tight_layout()
            for fig in figs_da:
                plt.figure(fig.number)
                pdf.savefig(layout="tight")
                plt.close(fig)

In [ ]:
from tampering_utils import plot_activation_function, inverse_sigmoid, hard_sigmoid, sinusoidal_sigmoid
import keras
import tensorflow as tf
class inverse_sigmoid2(keras.layers.Layer):
    def __init__(self, constant1=10, constant2=1, limits=[0, 0]):
        super(inverse_sigmoid2, self).__init__()
        self.constant1 = constant1
        self.constant2 = constant2
        self.limits = np.mean(limits)

    def sigmoid(self, x, constant):
        return 1/ (1 + (tf.exp(-constant * x)))

    def call(self, inputs):
        term_1 = self.sigmoid(-(inputs-self.limits), constant=self.constant1) * \
            (-0.6 + self.sigmoid(-(inputs-self.limits), constant=self.constant2))
        term_2 = self.sigmoid(inputs-self.limits, constant=self.constant1) * \
            (0.6 + self.sigmoid(-(inputs-self.limits), constant=self.constant2))
        outputs = term_1 + term_2
        return outputs

activation_functions = [inverse_sigmoid, hard_sigmoid, sinusoidal_sigmoid]
figs = []
for activation_function in activation_functions:
    values, _, fig, model = plot_activation_function(activation_function, return_model=True)
    figs.append(fig)

# model.summary()
# # input_seqs = values#np.arange(-10, 10, 0.1)
# model_in = values.reshape(-1, 1)
# model_in = tf.constant(model_in)
# print(model_in.shape)
# with tf.GradientTape() as tape:
#     tape.watch(model_in)
#     model_out = model(model_in)
#     grads = tape.gradient(model_out, model_in)
#     # print(grads)
# grads = grads.numpy()
# plt.plot(values, grads[:,0])
# plt.ylim([-1, 1])

# grads.shape


with PdfPages(path+'Figures/activation_functions.pdf') as pdf:
    for fig in figs:
        plt.figure(fig.number)
        pdf.savefig(layout='tight')

In [ ]:
x_tr_seq, y_train, x_val_seq, y_test, tokenizer, size_of_vocabulary = data
activations = [None, inverse_sigmoid, hard_sigmoid, sinusoidal_sigmoid]
activation_names = ["No Tampering", "Inverse Sigmoid", "Hard Sigmoid", "Sinusoidal Sigmoid"]

figures_all = []
for custom_dropout in custom_dropouts:
    print("dropout_value: ", custom_dropout)
    model = load_trained_model(path, size_of_vocabulary, data_name=data_name, 
                            num_classes=y_train.shape[1], custom_dropout=custom_dropout)
    model_unmasked = inference_model(model, include_softmax=False)
    min_threshold, max_threshold, fig = visualize_activations(model_unmasked, x_tr_seq[:100], figure=True, bins=100)
    figures_all.append(fig)
    
    # accuracies = []
    # for i in range(len(activations)):
    #     model_tampered = inference_model(model, tampering_activation=activations[i], 
    #                             tampering_limits=([min_threshold, max_threshold]))
    #     accuracies.append(model_tampered.evaluate(x_val_seq, y_test)[1])
    
with PdfPages(path+'Figures/logit_activations_'+data_name+'_'+str(max_len)+'.pdf') as pdf:
    for fig in figures_all:
        plt.figure(fig.number)
        pdf.savefig()

In [ ]:
def check(model_r):
    print(np.argmax(model_r.predict(x_val_seq[5:10]), axis=1), model_r.predict(x_val_seq[5:10]))
    print("==========================================")

print(np.mean([min_threshold, max_threshold]))
for model_r in ([model, model_unmasked, model_tampered]):
    check(model_r)